# Imports

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
import time
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import numpy as np

### Init Browser

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\kheat\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# KenPom Loop

In [3]:
years = list(np.arange(2016, 2022))


for year in years:
    clean_names = []
    seeds = []
    
    df = pd.DataFrame()
    
    url = f'https://kenpom.com/index.php?y={year}'
    browser.visit(url)
    time.sleep(0.5)
    
    html=browser.html
    soup = bs(html, 'lxml')
    
    dfs = pd.read_html(browser.html)
    df = dfs[0]
    df.columns = [x[1] for x in df.columns]
    df.Team[0][-2:]
    str.isnumeric(df.Team[0][-2:].strip())
    
    for _, row in df.iterrows():
        seed = row.Team[-2:].strip()

        if str.isnumeric(seed):
            clean_name = row.Team[:-2]

        else:
            clean_name = row.Team
            seed = ''

        clean_names.append(clean_name)
        seeds.append(seed)
  
    df["TeamName_Clean"] = clean_names
    df["Seed"] = seeds
    
    df.columns = ['Rk', 'Team', 'Conf', 'W-L', 'AdjEM', 'AdjO', 'AdjO_Rank', 'AdjD', 'AdjD_Rank',
       'AdjT', 'AdjT_Rank', 'Luck', 'Luck_Rank', 'SOS_AdjEM', 'AdjEM_Rank', 'SOS_OppO', 'OppO_Rank',
       'SOS_OppD', 'OppD_Rank', 'NCSOS_AdjEM', 'AdjEM_Rank', 'TeamName_Clean', 'Seed']

    df_kenPom = df.drop(['Team','W-L','AdjO_Rank', 'AdjD_Rank', 'AdjT_Rank','Luck_Rank','AdjEM_Rank', 
                         'OppO_Rank', 'OppD_Rank','AdjEM_Rank', 'Seed'], axis=1)

    df_kenPom = df_kenPom[['Rk', 'TeamName_Clean', 'AdjEM', 'AdjO', 'AdjD', 'AdjT', 'Luck', 'SOS_AdjEM', 
                           'SOS_OppO', 'SOS_OppD', 'NCSOS_AdjEM']]
    
    df_kenPom['Year'] = int(year)

    # Write each df to CSV
    df_kenPom.to_csv(f'Data/2021/kenPom_{year}.csv', index=False)


In [4]:
df_kenPom.head()

,Rk,TeamName_Clean,AdjEM,AdjO,AdjD,AdjT,Luck,SOS_AdjEM,SOS_OppO,SOS_OppD,NCSOS_AdjEM,Year
0,1,Gonzaga,37.38,126.4,89.0,74.3,0.018,8.75,106.3,97.5,6.12,2021
1,2,Baylor,30.53,122.3,91.8,68.2,0.048,10.67,107.0,96.3,-2.99,2021
2,3,Houston,30.30,118.5,88.2,64.7,0.000,8.36,104.7,96.4,2.13,2021
3,4,Michigan,29.29,118.6,89.3,66.9,0.029,15.70,111.0,95.3,2.57,2021
4,5,Illinois,28.87,117.6,88.8,70.5,0.022,18.00,112.0,94.0,10.21,2021


In [5]:
files = ['kenPom_2016.csv', 'kenPom_2017.csv', 'kenPom_2018.csv', 'kenPom_2019.csv', 'kenPom_2020.csv', 'kenPom_2021.csv']

csvs = []

for x in files:
    csvs.append(pd.read_csv(f'Data/2021/{x}'))

kenPom_Full = pd.concat(csvs).reset_index(drop=True)
kenPom_Full.to_csv('Data/2021/kenPom_Full.csv', index=False)